# XGBoost

A classificação por floresta aleatória é uma técnica de aprendizado de máquina que combina o poder da aleatoriedade com o poder da média para construir um modelo robusto de classificação. Ela opera criando múltiplas árvores de decisão durante o treinamento e fazendo previsões com base na maioria das previsões das árvores individuais.

## Principais Características:
- **Árvores de Decisão:** Cada árvore na floresta é construída a partir de uma amostra aleatória do conjunto de dados de treinamento. Isso ajuda a reduzir a correlação entre as árvores individuais, tornando o modelo mais robusto.

- **Amostragem Aleatória de Características:** Durante a construção de cada árvore, apenas um subconjunto aleatório das características é considerado para dividir em cada nó da árvore. Isso introduz mais diversidade nas árvores e reduz a probabilidade de overfitting.

- **Votação por Maioria:** Ao fazer previsões, cada árvore na floresta contribui com uma votação para determinar a classe final de um exemplo. A classe mais frequente entre todas as árvores é escolhida como a previsão final.

## Vantagens:
- **Robustez:** Averiguando várias árvores de decisão, o modelo é menos propenso a overfitting e tem uma melhor capacidade de generalização em dados de teste.

- **Manuseio de Dados Não Lineares e de Alta Dimensão:** Pode lidar eficazmente com conjuntos de dados com muitas características e interações complexas entre elas.

- **Fácil de Usar:** Requer pouca ou nenhuma sintonia de parâmetros e lida bem com dados ausentes.

## Limitações:
- **Interpretabilidade:** Às vezes, a floresta aleatória pode ser difícil de interpretar em comparação com modelos lineares simples.

- **Desempenho em Dados Escaláveis:** Para conjuntos de dados muito grandes, o treinamento de uma floresta aleatória pode se tornar computacionalmente caro.

## Aplicações:
- **Classificação:** Prever a classe de um exemplo com base em suas características.

A implementação da classificação por floresta aleatória no scikit-learn, utilizando RandomForestClassifier, oferece uma interface simples e poderosa para treinar e fazer previsões com este modelo.

# Parâmetros do `XGBoost`:

**Parâmetros Gerais:**

1. `booster`: Define o tipo de modelo a ser usado. Pode ser "gbtree" para árvore de decisão, "gblinear" para modelo linear ou "dart" para adição de dropout à árvore de decisão.

2. `verbosity`: Controla o nível de verbosidade ao imprimir mensagens. Pode ser 0 (silencioso), 1 (informações detalhadas) ou 2 (informações detalhadas e mensagens de depuração).

3. `nthread`: Número de threads a serem usados para rodar XGBoost. Se definido como -1, o máximo de threads disponíveis será usado.

**Parâmetros de Booster (Árvore):**

1. `eta` (ou `learning_rate`): Taxa de aprendizado. Controla a contribuição de cada árvore no modelo.

2. `gamma`: Mínima redução da perda necessária para fazer uma nova partição em um nó da árvore.

3. `max_depth`: Profundidade máxima de cada árvore. Maior profundidade pode levar a overfitting.

4. `min_child_weight`: Soma mínima dos pesos das amostras necessária em um nó para continuar dividindo.

5. `max_delta_step`: Limite para a atualização do peso de cada árvore. Pode ajudar a tornar o treinamento mais conservador.

6. `subsample`: Fração de observações a serem amostradas aleatoriamente para cada árvore. Reduzir pode evitar overfitting.

7. `colsample_bytree`: Fração de features a serem amostradas aleatoriamente para cada árvore.

8. `lambda` (ou `reg_lambda`): Parâmetro de regularização L2.

9. `alpha` (ou `reg_alpha`): Parâmetro de regularização L1.

10. `scale_pos_weight`: Controla o balanceamento de classes para classificação desequilibrada.

**Parâmetros de Treinamento:**

1. `num_boost_round`: Número de iterações de boosting (número de árvores a serem criadas).

2. `early_stopping_rounds`: Se a métrica de avaliação não melhorar por esse número de rodadas, o treinamento será interrompido.

3. `eval_metric`: A métrica de avaliação a ser usada.

**Parâmetros de Avaliação:**

1. `eval_set`: Conjunto de dados de validação a serem usados para avaliação durante o treinamento.

2. `verbose_eval`: Controla a verbosidade da saída durante a avaliação.

**Parâmetros de Regressão Linear (para `gblinear`):**

1. `lambda`: Parâmetro de regularização L2.

2. `alpha`: Parâmetro de regularização L1.

3. `lambda_bias`: Parâmetro de regularização adicional para o termo de viés.

**Parâmetros específicos do Dart:**

1. `sample_type`: Tipo de amostra a ser usada. Pode ser "uniform" para amostras uniformes ou "weighted" para amostras ponderadas.

2. `normalize_type`: Tipo de normalização a ser usada. Pode ser "tree" para normalização por árvore ou "forest" para normalização por floresta.

3. `rate_drop`: Fração das árvores a serem eliminadas aleatoriamente a cada iteração.

4. `skip_drop`: Probabilidade de que uma árvore seja eliminada aleatoriamente.

Importando bibliotecas

In [1]:
import gsidiag as gd
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable

from itertools import combinations
from itertools import cycle
import seaborn as sns



Definindo parâmetros para o uso do readDiag

In [3]:
#DIRdiag = "/home/patrick/readDiag/data"
DIRdiag = "/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout"
#D:\ftp1.cptec.inpe.br\pesquisa\das\joao.gerd\EXP18\GSI\dataout
varName = "amsua"
varType = "n15"
dateIni="2020010100" 
dateFin="2020010206" 
nHour = "6"          
vminOMA = -2.0       
vmaxOMA = 2.0        
vminSTD = 0.0        
vmaxSTD = 14.0       
Level = 1000
Lay = None           
SingleL = "All" 

datei = datetime.strptime(str(dateIni), "%Y%m%d%H")
datef = datetime.strptime(str(dateFin), "%Y%m%d%H")
dates = [dates.strftime('%Y%m%d%H') for dates in pd.date_range(datei, datef,freq="6H").tolist()]

print(dates)


['2020010100', '2020010106', '2020010112', '2020010118', '2020010200', '2020010206']


Encontrando e listando arquivos para serem usados

In [4]:
paths, pathsc = [], []

OuterL = "01"        
[paths.append(DIRdiag+"/"+dt+"/diag_amsua_n15_"+OuterL+"."+dt) for dt in dates]

OuterLc = "03"
[pathsc.append(DIRdiag+"/"+dt+"/diag_amsua_n15_"+OuterLc+"."+dt) for dt in dates]

print(paths)
print("")
print(pathsc)

['/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010100/diag_amsua_n15_01.2020010100', '/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010106/diag_amsua_n15_01.2020010106', '/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010112/diag_amsua_n15_01.2020010112', '/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010118/diag_amsua_n15_01.2020010118', '/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010200/diag_amsua_n15_01.2020010200', '/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010206/diag_amsua_n15_01.2020010206']

['/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010100/diag_amsua_n15_03.2020010100', '/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010106/diag_amsua_n15_03.2020010106', '/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010112/diag_amsua_n15_03.2020010112', '/mnt/d

Lendo arquivos listado usando o readDiag e concatenando os dados

In [5]:
# Define uma variável booleana 'read' como True para indicar que a leitura dos arquivos será realizada
read = True

# Verifica se 'read' é True para prosseguir com a leitura dos arquivos
if read:        
    # Inicializa uma lista vazia para armazenar os objetos gdf
    gdf_list = []
    # Imprime uma mensagem informando o tempo estimado necessário para a leitura dos arquivos
    print("")
    print("Aguarde, o tempo total estimado para a leitura dos arquivos é de "+
          str(int((float(len(paths))*20 )/60))+" minutos e "+
          str(int((float(len(paths))*20 )%60))+" segundos.")
    print("")
    # Itera sobre os caminhos dos arquivos e seus caminhos de configuração correspondentes
    for path, pathc in zip(paths,pathsc):
        # Imprime uma mensagem indicando o arquivo que está sendo lido
        print("Reading "+path)
        # Lê o arquivo usando a função read_diag do módulo gsidiag e armazena o objeto retornado em gdf
        gdf = gd.read_diag(path,pathc)
        # Adiciona o objeto gdf à lista gdf_list
        gdf_list.append(gdf)

    # Imprime a lista de objetos gdf lidos
    print(gdf_list)
    
    # Define uma string separadora para uso posterior
    separator = " ====================================================================================================="



Aguarde, o tempo total estimado para a leitura dos arquivos é de 2 minutos e 0 segundos.

Reading /mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010100/diag_amsua_n15_01.2020010100
 
>>> GSI DIAG <<<
 
Reading /mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010106/diag_amsua_n15_01.2020010106
 
>>> GSI DIAG <<<
 
Reading /mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010112/diag_amsua_n15_01.2020010112
 
>>> GSI DIAG <<<
 
Reading /mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010118/diag_amsua_n15_01.2020010118
 
>>> GSI DIAG <<<
 
Reading /mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010200/diag_amsua_n15_01.2020010200
 
>>> GSI DIAG <<<
 
Reading /mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010206/diag_amsua_n15_01.2020010206
 
>>> GSI DIAG <<<
 
[<gsidiag.__main__.read_diag object at 0x7f4d4a603c90>, <gsidiag.__main__.read_diag object at

In [6]:
# Inicialização de DataFrame vazio para armazenar dados concatenados
df_concatenado2 = pd.DataFrame()

# Iteração sobre objetos em gdf_list
for objeto in gdf_list:
        
    print(separator)
    print("Separando dados do arquivo" + str(objeto._diagFile))
    
    # Criação de dicionário de dados a partir do objeto   
    dados_dict = {
            'lat': objeto.obsInfo[varName].loc[varType].lat,
            'lon': objeto.obsInfo[varName].loc[varType].lon,
            'elev': objeto.obsInfo[varName].loc[varType].elev,
            'nchan': objeto.obsInfo[varName].loc[varType].nchan,
            'time': objeto.obsInfo[varName].loc[varType].time,
            'iuse': objeto.obsInfo[varName].loc[varType].iuse,
            'idqc': objeto.obsInfo[varName].loc[varType].idqc,
            'inverr': objeto.obsInfo[varName].loc[varType].inverr,
            'oer': objeto.obsInfo[varName].loc[varType].oer,
            'obs': objeto.obsInfo[varName].loc[varType].obs,
            'omf': objeto.obsInfo[varName].loc[varType].omf,
            'omf_nobc': objeto.obsInfo[varName].loc[varType].omf_nobc,
            'emiss': objeto.obsInfo[varName].loc[varType].emiss,
            'oma': objeto.obsInfo[varName].loc[varType].oma,
            'oma_nobc': objeto.obsInfo[varName].loc[varType].oma_nobc,
            'imp': objeto.obsInfo[varName].loc[varType].imp,
            'dfs': objeto.obsInfo[varName].loc[varType].dfs
            }
        
    # Conversão do dicionário em DataFrame
    df_objeto = pd.DataFrame(dados_dict)
        
    # Concatenação do DataFrame do objeto com o DataFrame concatenado
    df_concatenado2 = pd.concat([df_concatenado2, df_objeto], ignore_index=True)
    

Separando dados do arquivo/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010100/diag_amsua_n15_01.2020010100
Separando dados do arquivo/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010106/diag_amsua_n15_01.2020010106
Separando dados do arquivo/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010112/diag_amsua_n15_01.2020010112
Separando dados do arquivo/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010118/diag_amsua_n15_01.2020010118
Separando dados do arquivo/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010200/diag_amsua_n15_01.2020010200
Separando dados do arquivo/mnt/d/ftp1.cptec.inpe.br/pesquisa/das/joao.gerd/EXP18/GSI/dataout/2020010206/diag_amsua_n15_01.2020010206


In [7]:
# Remoção de linhas com valores NaN
df_concatenado2.dropna(inplace=True)

# Filtrando os canais sub-representados
df_concatenado2 = df_concatenado2[~df_concatenado2['nchan'].isin([1, 2, 3, 4, 5, 6, 15])]

print(df_concatenado2)

print(separator)


# Define as classes do target discretizado para uso posterior
classes = ['6','7','8','9','10','12','13']

              lat         lon         elev  nchan      time  iuse  idqc  \
6      -87.504501   19.704599  2735.045410    7.0  0.315833   1.0   0.0   
7      -87.504501   19.704599  2735.045410    8.0  0.315833   1.0   0.0   
8      -87.504501   19.704599  2735.045410    9.0  0.315833   1.0   0.0   
9      -87.504501   19.704599  2735.045410   10.0  0.315833   1.0   0.0   
11     -87.504501   19.704599  2735.045410   12.0  0.315833   1.0   0.0   
...           ...         ...          ...    ...       ...   ...   ...   
697642  69.849403  359.350891    -0.074034    8.0  1.566111   1.0   0.0   
697643  69.849403  359.350891    -0.074034    9.0  1.566111   1.0   0.0   
697644  69.849403  359.350891    -0.074034   10.0  1.566111   1.0   0.0   
697646  69.849403  359.350891    -0.074034   12.0  1.566111   1.0   0.0   
697647  69.849403  359.350891    -0.074034   13.0  1.566111   1.0   0.0   

          inverr       oer         obs       omf  omf_nobc     emiss  \
6       3.996653  0.250215 

In [9]:
# Use pivot_table() para pivotar os dados
df_pivot2 = df_concatenado2.pivot_table(index=('lat', 'lon'), columns='nchan',
                                        values=('obs', 'oma', 'omf','imp', 'dfs'), aggfunc='mean')

# Resetando o índice para manter 'linha' como uma coluna
df_pivot2.reset_index(inplace=True)

# Remoção de linhas com valores NaN
df_pivot2.dropna(inplace=True)



# Lista de colunas que contêm os valores de DFS para cada canal
cols_dfs = [('dfs', 7.0), ('dfs', 8.0), ('dfs', 9.0), ('dfs', 10.0), ('dfs', 12.0), ('dfs', 13.0)]

# Encontrar o canal com o maior valor de DFS em cada linha
df_pivot2['canal_maior_dfs'] = df_pivot2[cols_dfs].idxmax(axis=1).str[1]

# Encontre o nome da coluna com o maior valor em cada linha
max_dfs_column = df_pivot2['dfs'].max(axis=1)

# Crie uma nova coluna no DataFrame com a informação do maior dfs em cada linha
df_pivot2['maior_dfs'] = max_dfs_column




# Lista de colunas que contêm os valores de imp para cada canal
cols_imp = [('imp', 7.0), ('imp', 8.0), ('imp', 9.0), ('imp', 10.0), ('imp', 12.0), ('imp', 13.0)]

# Encontrar o canal com o maior valor de imp em cada linha
df_pivot2['canal_maior_imp'] = df_pivot2[cols_imp].idxmax(axis=1).str[1]

# Encontre o nome da coluna com o maior valor em cada linha
max_dfs_column = df_pivot2['imp'].max(axis=1)

# Crie uma nova coluna no DataFrame com a informação do maior dfs em cada linha
df_pivot2['maior_imp'] = max_dfs_column


# Visualize o DataFrame com a nova coluna
print(df_pivot2)

             lat         lon       dfs                                \
nchan                              7.0       8.0       9.0      10.0   
0     -87.524902  145.937302  0.035069  0.016915  0.007011  0.000284   
1     -87.523804  298.361603 -0.039069  0.027618 -0.034395 -0.001559   
2     -87.522499   20.437300 -0.163683 -0.183544 -0.051867  0.008408   
3     -87.518898  299.104401  0.041977  0.044107  0.013228  0.005730   
4     -87.518501  216.446106  0.056615  0.092010 -0.066644 -0.015587   
...          ...         ...       ...       ...       ...       ...   
46505  86.926300  149.013702 -0.010454 -0.049940 -0.043989 -0.086170   
46506  86.930099  231.354004  0.038024 -0.158125 -0.212718 -0.294858   
46507  86.963997  258.552002 -0.010932  0.003193 -0.000108 -0.000085   
46508  87.024101   79.549599  0.007427  0.008859  0.027579  0.031323   
46509  87.076797  260.605286 -0.007154 -0.073785  0.001847  0.096013   

                                imp            ...       omf   

In [10]:
print(df_pivot2.columns)

MultiIndex([(            'lat',   ''),
            (            'lon',   ''),
            (            'dfs',  7.0),
            (            'dfs',  8.0),
            (            'dfs',  9.0),
            (            'dfs', 10.0),
            (            'dfs', 12.0),
            (            'dfs', 13.0),
            (            'imp',  7.0),
            (            'imp',  8.0),
            (            'imp',  9.0),
            (            'imp', 10.0),
            (            'imp', 12.0),
            (            'imp', 13.0),
            (            'obs',  7.0),
            (            'obs',  8.0),
            (            'obs',  9.0),
            (            'obs', 10.0),
            (            'obs', 12.0),
            (            'obs', 13.0),
            (            'oma',  7.0),
            (            'oma',  8.0),
            (            'oma',  9.0),
            (            'oma', 10.0),
            (            'oma', 12.0),
            (            

In [11]:
# 1. Preparação dos Dados
X = df_pivot2[['obs', 'oma', 'omf', 'dfs', 'imp', 'lat', 'lon']]
y = df_pivot2[['canal_maior_dfs', 'canal_maior_imp','maior_dfs', 'maior_imp']]

# 2. Divisão dos Dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.iloc[:, :21])


              obs                                                              \
nchan        7.0         8.0         9.0         10.0        12.0        13.0   
38105  218.809998  212.399994  209.350006  213.199997  235.429993  247.740005   
40949  215.880005  210.250000  203.570007  202.630005  222.610001  245.380005   
5155   230.610001  228.820007  229.190002  232.220001  245.539993  256.540009   
21886  221.000000  209.660004  201.580002  208.690002  230.270004  239.389999   
6557   228.199997  226.320007  226.770004  229.130005  246.160004  258.559998   
...           ...         ...         ...         ...         ...         ...   
11305  222.220001  216.949997  217.119995  223.139999  242.020004  254.619995   
44881  214.210007  210.009995  205.979996  208.179993  230.160004  245.399994   
38268  229.169998  225.910004  224.789993  226.699997  234.979996  239.580002   
860    228.320007  228.770004  232.240005  236.039993  247.990005  260.730011   
15829  227.169998  215.80000

In [12]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

# Ordenando o DataFrame com base na coluna 'nchan'
df_concatenado2.sort_values(by='nchan', inplace=True)

# Convertendo 'nchan' para string para ser usado como qid
qid = df_concatenado2['nchan']

# Separando as features (dfs)
X = df_concatenado2[['lat', 'lon', 'obs', 'oma', 'omf']]  # Features (dfs)
y = df_concatenado2[['imp']]  # Target

thresholds = [-2, -1, 0, 1, 2]  # Defina os limites conforme necessário
y_classes = np.digitize(y, thresholds, right=True)

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test, qid_train, qid_test = train_test_split(
    X, y_classes, qid, train_size=0.8, shuffle=True, random_state=42
)

# Ordenando os conjuntos de treinamento pelo qid
train_sorted_idx = np.argsort(qid_train)
X_train = X_train.iloc[train_sorted_idx]
y_train = y_train[train_sorted_idx]
qid_train = qid_train.iloc[train_sorted_idx]

# Ordenando os conjuntos de teste pelo qid
test_sorted_idx = np.argsort(qid_test)
X_test = X_test.iloc[test_sorted_idx]
y_test = y_test[test_sorted_idx]
qid_test = qid_test.iloc[test_sorted_idx]

# Convertendo qid para inteiros
qid_train = qid_train.astype(int)
qid_test = qid_test.astype(int)

# Configurando os parâmetros do XGBoost Ranker
params = {
    'objective': 'rank:pairwise',  # Usando a função de perda NDCG para problemas de ranking
    'eval_metric': 'map',      # Métrica de avaliação durante o treinamento
    'tree_method': 'hist',      # Método de construção da árvore para treinamento mais rápido
    'learning_rate': 0.2,       # Taxa de aprendizado (ajuste conforme necessário)
    'max_depth': 6,             # Profundidade máxima das árvores
    'min_child_weight': 3,      # Peso mínimo da instância necessária em cada nó folha
    'subsample': 0.8,           # Proporção de instâncias de treinamento a serem amostradas para treinar cada árvore
    'colsample_bytree': 0.8,    # Proporção de características a serem amostradas para treinar cada árvore
    'gamma': 0.1,               # Parâmetro de regularização para evitar overfitting
    'n_estimators': 200         # Número de árvores (estimadores) no modelo
}

# Criando o objeto XGBoost Ranker
ranker = xgb.XGBRanker(**params)

# Treinando o modelo
ranker.fit(X_train, y_train, qid=qid_train.values,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            eval_group=[qid_train.value_counts().sort_index().values,
                        qid_test.value_counts().sort_index().values],
            early_stopping_rounds=10, verbose=True)

# Realizando previsões no conjunto de teste
preds = ranker.predict(X_test)

print(X_test)
print(y_test)
print(preds)

[0]	validation_0-map:0.99657	validation_1-map:0.99663
[1]	validation_0-map:0.99937	validation_1-map:0.99944
[2]	validation_0-map:0.99912	validation_1-map:0.99924
[3]	validation_0-map:0.99872	validation_1-map:0.99885


/home/patrick/anaconda3/envs/readDiag/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[4]	validation_0-map:0.99927	validation_1-map:0.99937
[5]	validation_0-map:0.99900	validation_1-map:0.99910
[6]	validation_0-map:0.99901	validation_1-map:0.99911
[7]	validation_0-map:0.99905	validation_1-map:0.99916
[8]	validation_0-map:0.99915	validation_1-map:0.99925
[9]	validation_0-map:0.99904	validation_1-map:0.99913
[10]	validation_0-map:0.99911	validation_1-map:0.99919
              lat         lon         obs       oma       omf
75801   68.613701  283.049194  221.429993  0.243796  0.326914
291666  24.273899  243.814804  227.020004  0.009374  0.310378
96756  -47.671600  332.006012  225.770004 -0.137224 -0.113129
144021  28.127100   65.620697  224.559998  0.154185 -0.117707
112281 -28.156401  337.455505  224.139999  0.061626  0.108615
...           ...         ...         ...       ...       ...
630147 -45.145599  163.353394  252.520004  0.578940  1.242548
210522  78.977203   12.440600  213.740005 -0.112373 -0.553824
352392  -4.372500   21.472200  238.050003  0.844087  1.650331
5

In [13]:
import pandas as pd

# Criar um dataframe com as previsões, qid e os dados originais
resultados = pd.DataFrame({'qid': qid_test, 'previsoes': preds})

# Agrupar por qid e calcular a média das previsões para cada canal
media_por_canal = resultados.groupby('qid').mean()

print(media_por_canal)

     previsoes
qid           
7     0.472130
8     0.453402
9     0.444778
10    0.456390
12    0.439923
13    0.455529


In [14]:
print(resultados['qid'].value_counts())

12    9410
9     9334
13    9315
10    9268
7     9257
8     9198
Name: qid, dtype: int64


In [15]:
print(X)

              lat         lon         obs       oma       omf
6      -87.504501   19.704599  231.679993 -0.099744 -0.077042
409431 -12.357200  190.132294  227.610001  0.119951  0.202303
115326 -50.188999   23.128300  226.759995  0.173446 -0.089976
409416 -12.182500  188.734207  228.860001  0.186428  0.289750
409401 -12.485000  187.313004  228.949997 -0.009565  0.095424
...           ...         ...         ...       ...       ...
468807 -10.036100  336.723206  238.100006 -0.855764  0.025855
91557   54.083599  311.948914  207.449997  0.145808  0.236507
468792 -11.142900  359.108307  238.619995 -0.222396  1.023522
468957  -8.250300  338.404114  239.470001 -0.007398  0.345495
697647  69.849403  359.350891  218.509995  0.047907  1.329919

[278908 rows x 5 columns]


In [16]:
print(y)

             imp
6       0.016040
409431 -0.106147
115326  0.087946
409416 -0.196777
409401 -0.036052
...          ...
468807  0.487408
91557  -0.023100
468792 -0.664931
468957 -0.079472
697647 -1.176385

[278908 rows x 1 columns]


In [17]:
print(qid)

6          7.0
409431     7.0
115326     7.0
409416     7.0
409401     7.0
          ... 
468807    13.0
91557     13.0
468792    13.0
468957    13.0
697647    13.0
Name: nchan, Length: 278908, dtype: float32
